# Model inference
This notebook is used to load saved model weights and run inference on sample images

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pathlib import Path
import tensorflow as tf
from keras import layers
from keras import ops
from keras import Model
import keras

target_shape = (75, 75)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Preprocessed dataset saved in drive to read directly

anchor_path = '/content/drive/MyDrive/celebA Dataset/Matching_triplets/anchor_image.npy'
pos_path = '/content/drive/MyDrive/celebA Dataset/Matching_triplets/pos_image.npy'
neg_path = '/content/drive/MyDrive/celebA Dataset/Matching_triplets/neg_image.npy'
gender_path = '/content/drive/MyDrive/celebA Dataset/Matching_triplets/anchor_gender.npy'
age_path = '/content/drive/MyDrive/celebA Dataset/Matching_triplets/anchor_age.npy'
saved_model_dir = '/content/drive/MyDrive/celebA Dataset/weights'

# anchor_path = '/content/drive/MyDrive/Depi/Project/Model_training/celebA Dataset/Matching_triplets/anchor_image.npy'
# pos_path = '/content/drive/MyDrive/Depi/Project/Model_training/celebA Dataset/Matching_triplets/pos_image.npy'
# neg_path = '/content/drive/MyDrive/Depi/Project/Model_training/celebA Dataset/Matching_triplets/neg_image.npy'
# gender_path = '/content/drive/MyDrive/Depi/Project/Model_training/celebA Dataset/Matching_triplets/anchor_gender.npy'
# age_path = '/content/drive/MyDrive/Depi/Project/Model_training/celebA Dataset/Matching_triplets/anchor_age.npy'

# saved_model_dir = '/content/drive/MyDrive/Depi/Project/Model_training/celebA Dataset/weights'

In [ ]:
@keras.saving.register_keras_serializable()
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, test):
        at_distance = ops.sum(tf.square(anchor - test), -1)     # distance between anchor and test images
        return at_distance


## Load model from saved .keras file

In [ ]:
# Load model weight and pass an image for inference
complete_model = tf.keras.models.load_model(os.path.join(saved_model_dir, f'model_epoch_30.keras'))
# Remove the negative image branch from the model
inference_model = Model(complete_model.inputs[:2],
                        complete_model.outputs[:3])

## Run inference on a sample image

In [ ]:
# Load an image from dataset
anchor_images = np.load(anchor_path)
test_images = np.load(pos_path)
test_images_neg = np.load(neg_path)
age_all = np.load(age_path)
gender_all = np.load(gender_path)

In [ ]:
def inference_preprocess(image):
    """
    Preprocess the input image by resizing it to the target shape.
    """
    image = tf.image.resize(image, target_shape)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Ensure float32
    image = tf.expand_dims(image, axis=0, name=None)
    return image

In [ ]:
test_idx = random.randint(0, len(anchor_images))

anchor_image = inference_preprocess(anchor_images[test_idx])
test_image = inference_preprocess(test_images[test_idx])
test_image_neg = inference_preprocess(test_images_neg[test_idx])

In [ ]:
gender, age, distance = inference_model([anchor_image, test_image_neg])

In [ ]:
age_label = 'Young' if age > 0.5 else 'Not young'
gender_label = 'Male' if gender > 0.5 else 'Female'
verification = 1 if distance > 1 else 1

gender_label, age_label, verification